In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
folder_path = '/content/drive/MyDrive/pho-bert'
import os
os.listdir(folder_path)

['VnCoreNLP', 'test_vi.json', 'val_vi.json', 'train_vi.json', 'final_model']

In [3]:
!pip install transformers
!pip install torch
!pip install vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645952 sha256=bdd976a6396adba5f81de5a5bab824fbaca45d99622831581010df4dca4e3b5f
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [4]:
from transformers import BertTokenizer, BertForPreTraining, BertForQuestionAnswering, BertModel, BertConfig
from transformers import XLMRobertaForQuestionAnswering, XLMRobertaTokenizer
import torch
import torch.nn as nn
from transformers.data.metrics.squad_metrics import compute_predictions_log_probs, compute_predictions_logits, squad_evaluate
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor, SquadProcessor, SquadExample
from transformers import AutoModel, AutoTokenizer
from transformers import RobertaForQuestionAnswering, RobertaConfig, WEIGHTS_NAME

In [8]:
phobert = AutoModel.from_pretrained("vinai/phobert-large")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
model = RobertaForQuestionAnswering(phobert.config).from_pretrained("vinai/phobert-large")

Some weights of the model checkpoint at vinai/phobert-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-large were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/pho

In [10]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("/content/drive/MyDrive/pho-bert/VnCoreNLP-1.1.1/VnCoreNLP-1.1.1/VnCoreNLP-1.1.1.jar",port=9000, annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g') 
def VnCoreNLP1(X_train):
  X_train = annotator.tokenize(X_train)
  p = ''
  for i in range(0, len(X_train)):
    for x in X_train[i]:
      w = ''
      w = w + x + ' '
      p+=w
  p=p.replace(" ,",",")
  p=p.replace(" .",".")
  p=p.replace("( ","(")
  p=p.replace(" )",")")
  p=p.replace(' " ',' "')
  p=p.replace("* ","*")
  p=p.replace(" ;",";")
  p=p.replace(" ?","?")
  p=p.replace(" {","{")
  p=p.replace("{ ","{")
  p=p.replace(" }","}")
  p=p.replace("} ","}")
  p=p.replace(" /","/")
  p=p.replace("/ ","/")
  return p

In [11]:
def checkAns(answer, context_text):
  ans=context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])]
  l=ans.replace("_"," ")
  if(l == answer["text"]):
    return ans, answer["answer_start"]
  else:
    firstWord=answer["text"].split(' ')[0]
    lenWord=len(firstWord)
    for i in range(1,8):
      try:
        position_1=answer["answer_start"]-i
        word=context_text[position_1: position_1+ lenWord]
        if(firstWord == word):
          return context_text[position_1: position_1+ len(answer["text"])],position_1
        position_2=answer["answer_start"]+i
        word=context_text[position_2: position_2+ lenWord]
        if(firstWord == word):
          return context_text[position_2: position_2+ len(answer["text"])],position_2
      except:
        return ans, answer["answer_start"]
    return ans, answer["answer_start"]

In [12]:
def checkTempAns(answer, context_text):
  firstWord=answer["text"].split(' ')[0]
  lenWord=len(firstWord)
  for i in range(1,8):
    try:
      position_1=answer["answer_start"]-i
      word=context_text[position_1: position_1+ lenWord]
      if(firstWord == word):
        return context_text[position_1: position_1+ len(answer["text"])],position_1
      position_2=answer["answer_start"]+i
      word=context_text[position_2: position_2+ lenWord]
      if(firstWord == word):
        return context_text[position_2: position_2+ len(answer["text"])],position_2
    except:
      return context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])], answer["answer_start"]
  return context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])], answer["answer_start"]

In [13]:
from tqdm import tqdm
class VnCoreSquad(SquadProcessor):
  train_file = "train-v1.1.json"
  dev_file = "dev-v1.1.json"
  def _create_examples(self, input_data, set_type):
        is_training = set_type == "train"
        examples = []
        demso=0
        for entry in tqdm(input_data):
            title = entry["title"] 
            for paragraph in entry["paragraphs"]:
                context_text = VnCoreNLP1(paragraph["context"])
                for qa in paragraph["qas"]:
                    qas_id = qa["id"]
                    question_text = VnCoreNLP1(qa["question"])
                    start_position_character = None
                    answer_text = None
                    answers = []

                    is_impossible = qa.get("is_impossible", False)
                    if not is_impossible:
                        if is_training:
                            answer = qa["answers"][0]
                            answer_text, start_position_character=checkAns(answer, context_text)
                            qa["answers"][0].update({"text": answer_text})
                            qa["answers"][0].update({"answer_start": start_position_character})
                        else:
                            for k in range(0, len(qa["answers"])):
                              ans, start_position=checkTempAns(qa["answers"][k],context_text)
                              qa["answers"][k].update({"text": ans})
                              qa["answers"][k].update({"answer_start": start_position})
                              
                            answers = qa["answers"]
                    example = SquadExample(
                        qas_id=qas_id,
                        question_text=question_text,
                        context_text=context_text,
                        answer_text=answer_text,
                        start_position_character=start_position_character,
                        title=title,
                        is_impossible=is_impossible,
                        answers=answers,
                    )
                    examples.append(example)
        return examples
processor = VnCoreSquad()

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_examples = processor.get_train_examples('','/content/drive/MyDrive/pho-bert/train_vi.json')
dev_examples = processor.get_dev_examples('','/content/drive/MyDrive/pho-bert/val_vi.json')

100%|██████████| 13/13 [00:13<00:00,  1.04s/it]


In [ ]:
from transformers.data.processors.squad import squad_convert_examples_to_features

In [ ]:
train_features, train_dataset = squad_convert_examples_to_features(train_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = True,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

add example index and unique id: 100%|██████████| 21187/21187 [00:00<00:00, 811994.87it/s]


In [ ]:
del train_examples

In [5]:
from transformers.data.processors.squad import squad_convert_examples_to_features

In [ ]:
dev_features, dev_dataset = squad_convert_examples_to_features(dev_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

add example index and unique id: 100%|██████████| 2886/2886 [00:00<00:00, 444897.14it/s]


In [ ]:
import torch

def to_list(tensor):
    return tensor.detach().cpu().tolist()


In [17]:

import os
def evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features):
    eval_sampler = SequentialSampler(dev_dataset)
    eval_dataloader = DataLoader(dev_dataset, sampler=eval_sampler, batch_size=40)
    all_results = []
#     start_time = timeit.default_timer()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
#                "token_type_ids": batch[2],
            }
            example_indices = batch[3]
            outputs = model(**inputs)
            output_model = torch.cat((outputs.start_logits, outputs.end_logits), dim=1) # concatenate start_logits and end_logits along batch dimension
        for i, example_index in enumerate(example_indices):
            eval_feature = dev_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            output = outputs.start_logits[i], outputs.end_logits[i]
            if isinstance(output, tuple):  # regular logits
                start_logits, end_logits = output
            else:  # top k logits
                start_logits, end_logits, _, _ = output.chunk(4)
            result = SquadResult(unique_id, start_logits.detach().cpu().numpy(), end_logits.detach().cpu().numpy())
            all_results.append(result)
            # all_results.append(RawResult(unique_id=feature.unique_id, start_logits=start_logits.astype(np.float64), end_logits=end_logits.astype(np.float64)))

    
    # output_prediction_file = os.path.join("./", "predictions_{}.json".format(""))
    # output_nbest_file = os.path.join("./", "nbest_predictions_{}.json".format(""))
    # output_null_log_odds_file = os.path.join("./", "null_odds_{}.json".format(""))
    predictions = compute_predictions_logits(
            dev_examples,
            dev_features,
            all_results,
            20,
            128,
            False,
          None,
          None,
          None,
            True,
            False,
            0.0,
            tokenizer,
        )
    results = squad_evaluate(dev_examples, predictions)
    return results

In [ ]:
model_2 = None
tokenizer_2 = None

In [6]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import trange, tqdm
device = torch.device('cuda')

In [ ]:
num_epochs = 4
tb_writer = SummaryWriter()
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=16)
t_total = len(train_dataloader) // 1 * num_epochs

output_dir = os.path.join('/content/drive/MyDrive/pho-bert/', 'final_model')
no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=814, num_training_steps=t_total
)

device = torch.device('cuda')

model.to(device)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0

model.zero_grad()
train_iterator = trange(
    epochs_trained, int(num_epochs), desc="Epoch", disable=-1 not in [-1, 0]
)

from functools import partial
tqdm = partial(tqdm, position=0, leave=True)
max_f1 = 0
patient = 0
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
    for step, batch in enumerate(epoch_iterator):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        global_step += 1

        if global_step % 5000 == 0:
            print(" global_step = %s, average loss = %s", global_step, tr_loss/global_step)
    model.eval()
    model_2 = model
    tokenizer_2 = tokenizer
    results = evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features)
    print("exact: " + str(results['exact']))
    print(results['f1'])
    if results['f1'] >= max_f1:
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(output_dir)
        max_f1 = results['f1']
        patient = 0
    else:
        patient = patient+1
    if patient == 3:
        break

del model

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Evaluating: 100%|██████████| 89/89 [02:44<00:00,  1.84s/it]


exact: 34.82328482328482
51.47512247929256


Evaluating: 100%|██████████| 89/89 [02:44<00:00,  1.85s/it]


exact: 37.35273735273735
53.2766638037215


Epoch:  75%|███████▌  | 3/4 [3:27:31<1:09:13, 4153.41s/it]

exact: 35.585585585585584
52.65181294017144


Iteration:   3%|▎         | 49/1645 [01:58<1:04:26,  2.42s/it]

In [14]:
model_2 = RobertaForQuestionAnswering(phobert.config).from_pretrained("/content/drive/MyDrive/pho-bert/final_model").to(device)
tokenizer_2 = AutoTokenizer.from_pretrained("vinai/phobert-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
test_examples = processor.get_dev_examples('', '/content/drive/MyDrive/pho-bert/test_vi.json')
test_features, test_dataset = squad_convert_examples_to_features(test_examples, 
                                                       tokenizer_2, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

add example index and unique id: 100%|██████████| 4384/4384 [00:00<00:00, 570412.85it/s]


In [18]:
results = evaluate(model_2, tokenizer_2, test_dataset, test_examples, test_features)
for key, value in results.items():
    print(key, value)

Evaluating: 100%|██████████| 130/130 [03:45<00:00,  1.73s/it]


exact 34.215328467153284
f1 50.46814053845613
total 4384
HasAns_exact 51.08991825613079
HasAns_f1 75.35842238439771
HasAns_total 2936
NoAns_exact 0.0
NoAns_f1 0.0
NoAns_total 1448
best_exact 36.792883211678834
best_exact_thresh 0.0
best_f1 50.51376097641239
best_f1_thresh 0.0


In [ ]:
# Define context and question
context = ''' Chủ tịch Hồ Chí Minh (tên lúc nhỏ là Nguyễn Sinh Cung, tên khi đi học là Nguyễn Tất Thành, trong nhiều năm hoạt động cách mạng trước đây lấy tên là Nguyễn Ái Quốc), sinh ngày 19-5-1890 ở làng Kim Liên, xã Nam Liên (nay là xã Kim Liên), huyện Nam Đàn, tỉnh Nghệ An và mất ngày 2-9-1969 tại Hà Nội.
Người sinh ra trong một gia đình: Bố là một nhà nho yêu nước, nguồn gốc nông dân; mẹ là nông dân; chị và anh đều tham gia chống Pháp và bị tù đày.
Trung ương Đảng và Chủ tịch Hồ Chí Minh đề ra hai nhiệm vụ chiến lược của cách mạng Việt Nam là tiến hành cách mạng xã hội chủ nghĩa và xây dựng chủ nghĩa xã hội ở miền Bắc, đồng thời đấu tranh giải phóng miền Nam, thực hiện thống nhất nước nhà, hoàn thành cách mạng dân tộc dân chủ nhân dân trong cả nước'''
question = "Hồ Chí Minh được bầu làm gì?"

# Tokenize inputs
inputs = tokenizer_2.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# Move inputs tensor to device
inputs = {key: value.to(torch.device('cuda')) for key, value in inputs.items()}

# get start and end logits for answer
start_logits, end_logits = model_2(**inputs, return_dict=False)

# find the answer
start_idx = torch.argmax(start_logits) + 1
end_idx = torch.argmax(end_logits) + 2
answer = tokenizer_2.convert_tokens_to_string(tokenizer_2.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))

print("Answer:", answer)